## Problem 1 - 1D unconstrained optimization

### (a). Write a functioning one-dimensional black box unconstrained function optimizer.

In [253]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

Here we will use the Fibonacci method for optimization. Our function will return a couple of useful values:
* $x^*$
* $f(x^*)$
* number of function calls for $f(x)$

In [247]:
def fib_min(f, x_1, x_4, n):
    I_0 = np.abs(x_4 - x_1)  # Initial interval of uncertainty
    c = (np.sqrt(5) - 1) / 2
    s = (1 - np.sqrt(5)) / (1 + np.sqrt(5))
    alpha = (c*(1 - s**n)) / (1 - s**(n+1))
    x_3 = alpha*x_4 + (1 - alpha)*x_1
    f_3 = f(x_3)
    
    for i in range(1, n+1):
        x_2 = 0.01*x_1 + 0.99*x_3 if i == n else alpha*x_1 + (1 - alpha)*x_4
        
        f_2 = f(x_2)
        if f_2 < f_3:
            x_4 = x_3
            x_3 = x_2
            f_3 = f_2
        else:
            x_1 = x_4
            x_4 = x_2
        alpha = (c*(1 - s**(n-i))) / (1 - s**(n-i+1))
        
    x_star = np.median([x_1, x_2, x_3, x_4])
    f_star = f(x_star)
    ncalls = f.called()
    f.reset()
    return(x_star, f_star, ncalls)

### (b). Test code of part (a). on two different 1D unconstrained optimization problems, with optimization starting points drawn from a probability distribution.

First, let's create a little helper function to keep track of the number of function calls

In [248]:
# Helper function to count function calls
def count(f):
    def call(*args,**kargs):
        call.calls += 1
        return f(*args,**kargs)

    call.calls = 0
    def reset(): call.calls = 0 
    call.reset = reset
    call.called = lambda : call.calls
    return call

Let's make up some functions, $f(x)$, to optimize:
* $f_{1}(x) = x^2 + 2x \:$  (quadratic)
* $f_{2}(x) = x^4 - 13x^2 + 36 \:$  (non-quadratic)

In [249]:
@count
def f_1(x):
    return x**2 + 2*x

@count
def f_2(x):
    return x**4 - 13*x**2 + 36

We now need to determine the starting points for our optimizer and determine the number of iterations or stopping conditions. For simplicity, let us obtain $x^*$ such that $f(x^*)$ is within $1\%$ of the true optimum.

$$\begin{aligned}
\frac{I_n}{2I_0} &\leq \frac{1}{100}\\
\frac{I_n}{I_0}  &\leq \frac{1}{50}\\
\frac{1}{F_n}    &\leq \frac{1}{50}\\
F_n              &\geq 50
\end{aligned}$$

Let us now obtain $n$ from the fibonacci sequence using Binet's formula

In [250]:
def fib(n):
    return (((1 + np.sqrt(5)) / 2)**n - ((1 - np.sqrt(5)) / 2)**n) / np.sqrt(5)

for n in range(2, 100):
    fib_n = fib(n)
    if fib_n >= 50:
        print('n = {}\nfib_n = {}'.format(n, fib_n))
        break

n = 10
fib_n = 55.000000000000014


Let us now create some random samples for our starting points. We'll draw from uniform distribution and sample from the interval $[-100, 100)$

In [251]:
# Set random seed
np.random.seed(42)

# Create samples
samples = 200 * np.random.random_sample((10, 2)) - 100
samples

array([[-25.09197623,  90.14286128],
       [ 46.39878836,  19.73169684],
       [-68.79627191, -68.80109593],
       [-88.38327757,  73.23522915],
       [ 20.22300235,  41.61451556],
       [-95.88310114,  93.98197043],
       [ 66.48852816, -57.53217786],
       [-63.63500656, -63.31909803],
       [-39.15155141,   4.95128633],
       [-13.61099627, -41.75417196]])

In [252]:
n = 10
f1_results = [fib_min(f_1, s[0], s[1], n) for s in samples]
f2_results = [fib_min(f_2, s[0], s[1], n) for s in samples]

In [238]:
%%time
x_star, f_star, fcalls = fib_min(f_1, -3, 4, 10)

-0.997980053024 -0.998916184825 -0.998916184825 -0.998863779826
[-0.99891618482514843, -0.99891618482514843, -0.99886377982577956, -0.99798005302360815]
CPU times: user 3.27 ms, sys: 150 µs, total: 3.42 ms
Wall time: 2.12 ms


In [212]:
print(fcalls)

12


In [213]:
print('optimal x: {}\noptimal f: {}'.format(x_star, f_star))

optimal x: -1.0329213483146065
optimal f: -0.9989161848251484


In [214]:
acc = (-1 - f_star) / -1

In [210]:
print(acc*100)

7.16513459675e-06


In [143]:
samples

array([[  1.94972259,  11.94734344],
       [-79.02933576,  80.69103427],
       [-17.0294361 ,  29.91113313],
       [ 19.12171141, -12.72359808],
       [ 32.53363108,  57.90843627],
       [ 70.67970401, -65.63926054],
       [-51.68873599,  55.21598998],
       [-66.86649278,   7.77447779],
       [ -0.3172908 , -41.03367147],
       [  6.9478492 , -70.39627008]])

11.947343443286783